<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    GDP Intermediate Revisions and Horizon Datasets
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h3>
Documentation
<br>
____________________
<br>
</h3>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    This 
    <span style="color: rgb(0, 65, 75);">jupyter notebook</span>
    provides a step-by-step guide to <b>data building</b> regarding the project <b>'Revisiones y sesgos en las estimaciones preliminares del PBI en el Perú'</b>. This guide covers the creation of GDP mid-term revision dataset for each sector. A key step is the construction at par of what we will call “The ‘t+h’ structure”. This dataset is similar to that of the GDP growth vintages by sector, but instead of growth rate values, it contains values of type “t+h”, where h indicates how many months have passed since the preliminary growth rate was first published; that is, this jupyter notebook also covers the creation of vintages datasets of growth rates associated with a horizon (<b>h</b>).
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;line-height: 1.5;">
<span style="font-size: 34px;">&#128452;</span> The 't+h' structure should be available for all sectors and frequencies.
    <br>
    <span style="font-size: 24px;">&#8987;</span> Available since <b>1994-2024</b> (Table 1) and since <b>1997-2024</b> (Table 2). 
    <br>
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        1. Economic sector selector</a>
    <br>
    <a href="#2" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        2. Create horizon datasets</a>
    <br>
    <a href="#2.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#2.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.2. Creating horizon dataset step by step.</a> 
    <br>
    <a href="#3" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        3. Create intermediate revisions datasets</a>
    <br>
    <a href="#3.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.1. Creating intermediate revisions dataset step by step.</a>
    <br>
    <a href="#3.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.2. Clean-up intermediate revisions dataset (last version to be loaded to SQL).</a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"><span style="font-size: 24px;">&#x2709;</span>
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# POSTGRESSQL
import os
from sqlalchemy import create_engine

# HORIZON DATASETS
import pandas as pd
import numpy as np
import re


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [2]:
def create_sqlalchemy_engine():
    """
    Function to create an SQLAlchemy engine using environment variables.
    
    Returns:
        engine: SQLAlchemy engine object.
    """
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')  # Get the SQL user from environment variables
    password = os.environ.get('CIUP_SQL_PASS')  # Get the SQL password from environment variables
    host = os.environ.get('CIUP_SQL_HOST')  # Get the SQL host from environment variables
    port = 5432  # Set the SQL port to 5432
    database = 'gdp_revisions_datasets'  # Set the database name 'gdp_revisions_datasets' from SQL

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    return engine

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>gdp_inter_revisions_datasets_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok.<div/>

In [3]:
from gdp_inter_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Economic sector and data frequency selector</span></h1>

<div id="select">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">1.1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Economic sector</span></h2>

In [4]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

Selected economic sector: gdp


<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">1.2. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Frequency</span></h2>

In [5]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: monthly


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create horizon datasets</span></h1>

<div id="2.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [6]:
# Connect to SQL
engine = create_sqlalchemy_engine()

# SQL Query
query = f"SELECT * FROM {sector}_{frequency}_growth_rates;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [8]:
# Read growth rates dataset as DataFrame
globals()[f'{sector}_{frequency}_growth_rates'] = pd.read_sql(query, engine)
vintages = globals()[f'{sector}_{frequency}_growth_rates']
vintages_df = vintages.copy()
vintages_df.head(10)

,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,2013,01,2013-01-04,5.9,5.3,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,5.9,5.3,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2.2.">
   <!-- Contenido de la celda de destino -->
</div>

<div id="2.3.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating horizon dataset step by step
    </span>
    </h2>

<div id="steps">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Replace decimal values by “t+h” values only in the rows representing a new rung</a>
    <br>
    <a href="#step-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Concatenate first 3 columns: year, date, id_ns </a>
    <br>
    <a href="#step-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Convert columns to string type</a>
    <br>
    <a href="#step-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values </a>
    <br>
    <a href="#step-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file </a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
   <span>Set key variables to fill the vintages with 't+h' values</span>
  </div>

In [9]:
# Call the function to show the h_initial window
h_initial = show_h_initial_window()
print("Selected h_initial:", h_initial)

Selected h_initial: 3


In [10]:
# Call the function to show the start_row window
start_row = show_start_row_window()
print("Selected start_row:", start_row)

Selected start_row: 0


In [11]:
# Define the mapping of frequencies to h_counter values
frequency_mapping = {
    'monthly': 1,
    'quarterly': 3,
    'annual': 12
}

# Get the appropriate h_counter value based on the selected frequency
h_counter = frequency_mapping.get(frequency)
print("Selected h_counter:", h_counter)

Selected h_counter: 1


<div id="step-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Replace decimal values by “t+h” values only in the rows representing a new rung</span>
  </div>

In [12]:
horizon = replace_horizon(vintages_df.iloc[:, 3:], start_row, h_initial, h_counter)
horizon_df = horizon.copy()
horizon_df.head(10)

,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.9,5.3,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

<div id="step-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Concatenate first 3 columns: year, date, id_ns</span>
  </div>

In [13]:
# Get the first three columns of the original DataFrame
first_3_columns = vintages_df.iloc[:, :3]

# Concatenate the first three columns with h_{sector}_{frequency}_growth_rates
horizon_df = pd.concat([first_3_columns, horizon_df], axis=1)
horizon_df.head(10)

,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,2013,01,2013-01-04,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,5.9,5.3,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,NaN,NaN,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,NaN,NaN,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

<div id="step-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Convert columns to string type</span>
  </div>

In [14]:
horizon_df = columns_str(horizon_df)
horizon_df.head(10)

,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,2013,01,2013-01-04,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2013,02,2013-01-11,5.9,5.3,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2013,03,2013-01-18,,,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,2013,04,2013-01-25,,,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2013,05,2013-02-01,,,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,2013,06,2013-02-08,,,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,2013,07,2013-02-15,,,5.1,6.0,5.5,6.9,5.7,4.5,7.0,7.3,7.4,6.3,5.9,6.7,6.8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,2013,08,2013-02-22,,,,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,2013,09,2013-03-01,,,,6.0,5.5,6.9,5.7,4.6,7.1,7.4,7.5,6.5,6.3,6.8,6.8,4.3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,2013,10,2013-03-08,,,,6.0,5.5,6.9,5.7,4.6,7.1,7.4,7.5,6.5,6.3,6.8,6.8,4.3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


<div id="step-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values
  </div>

In [15]:
horizon_df = replace_horizon_1(horizon_df)
horizon_df.head(10)

,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,2013,01,2013-01-04,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2013,02,2013-01-11,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2013,03,2013-01-18,,,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,2013,04,2013-01-25,,,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2013,05,2013-02-01,,,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,2013,06,2013-02-08,,,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,2013,07,2013-02-15,,,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,2013,08,2013-02-22,,,,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,2013,09,2013-03-01,,,,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,2013,10,2013-03-08,,,,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


<div id="step-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file
  </div>

In [16]:
#def export_to_excel(df, filename):
#    # Exportar el DataFrame como un archivo Excel
#    df.to_excel(filename, index=False)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create intermediate revisions datasets</span></h1>

<div id="3.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating intermediate revisions dataset step by step
    </span>
    </h2>

<div id="steps-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary with the row indices and their t+h values</a>
    <br>
    <a href="#st-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Computing intermediate revisions </a>
    <br>
    <a href="#st-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Transpose the intermediate revisions dataset</a>
</div>

<div id="st-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary with the row indices and their t+h values</span>
  </div>

In [17]:
dictionary = get_last_index_h(horizon_df)

<div id="st-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Computing intermediate revisions </span>
  </div>

In [18]:
inter_revisions = computing_inter_revisions(vintages_df, dictionary)
inter_revisions_df = inter_revisions.copy()
inter_revisions_df

,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.1,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0,0.2,1.7,0.0,-0.3,0.0,0.0,0.5,0.0,0.0,0.1,0.0,0.0,-0.1,0.0,0.0,0.1,0.0,0.0,-0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.0,-0.1,0.0,0.0,0.1,0.0,0.0,0.2,0.05,0.0,-0.02,0.0,0.0,0.11,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,-0.1,0.0,0.0,0.1,0.0,0.0,-0.1,0.0,0.0,-0.1,0.0,0.0,-0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.3,0.0,0.0,0.4,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.2,0.1,0.0,-0.1,-0.2,0.0,0.1,0.0,0.0,0.1,2.4,1.7,-0.2,-0.3,0.0,0.4,0.5,0.0,0.1,0.1,0.0,0.0,-0.1,0.0,-0.2,0.1,0.0,-0.1,-0.1,0.0,0.0,0.0,0.0,0.4,0.1,0.0,0.0,0.2,0.0,0.2,-0.1,0.0,-0.1,0.1,0.0,0.1,0.16,0.05,0.12,-0.02,0.0,0.03,0.11,0.0,0.2,0.2,0.0,0.6,0.2,0.0,0.1,-0.1,0.0,0.0,0.1,0.0,0.2,-0.1,0.0,0.4,-0.1,0.0,0.0,-0.1,0.0,0.2,0.1,0.0,0.0,0.0,0.0,0.3,0.1,0.0,0.0,0.1,0.0,0.6,0.1,0.0,0.1,0.1,0.0,0.5,0.3,0.0,0.0,0.4,0.0,0.9,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0.0,0.2,0.0,0.0,0.3,0.2,0.0,0.1,0.1,0.0,0.1,0.1,0.0,0.1,0.1,0.0,0.1,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.2,0.1,-0.6,-0.1,-0.2,0.0,0.1,0.0,0.0,0.7,2.4,1.9,-0.2,-0.3,0.5,0.4,0.5,0.0,0.1,0.1,0.0,0.0,-0.1,0.0,-0.2,0.1,0.0,-0.1,-0.1,0.0,0.0,-0.2,0.4,0.4,0.1,0.0,0.0,0.2,0.0,0.2,-0.1,0.0,-0.1,0.1,0.2,0.13,0.16,0.14,0.12,-0.02,0.19,0.03,0.11,0.0,0.2,0.2,0.4,0.6,0.2,0.0,0.1,-0.1,-0.1,0.0,0.1,0.1,0.2,-0.1,0.2,0.4,-0.1,0.1,0.0,-0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.2,0.3,0.1,0.0,0.1,0.1,-0.4,0.6,0.1,0.2,0.1,0.1,0.7,0.5,0.3,0.1,0.0,0.4,1.8,0.9,0.5,0.2,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.0,0.0,0.3,0.2,0.1,0.2,0.0,0.4,0.3,0.2,0.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.2,-0.1,-0.6,-0.1,-0.3,0.0,0.1,0.1,0.3,0.7,2.7,1.9,-0.2,-0.2,0.5,0.4,0.5,0.0,0.1,0.0,0.0,0.0,-0.1,0.0,-0.2,0.1,0.0,-0.1,-0.1,0.0,0.0,0.1,0.4,0.4,0.1,0.0,0.0,0.2,0.0,0.2,-0.1,0.0,-0.1,0.2,0.16,0.13,0.21,0.14,0.12,-0.04,0.19,0.03,0.01,0.0,0.2,0.5,0.4,0.6,0.2,0.0,0.1,-0.3,-0.1,0.0,0.2,0.1,0.2,0.1,0.2,0.4,-0.1,0.0,0.0,0.0,0.1,0.2,0.2,0.0,0.0,0.2,0.2,0.3,0.1,0.0,0.1,0.1,-0.4,0.6,0.5,0.2,0.1,0.5,0.7,0.5,0.3,0.1,0.0,0.7,1.8,0.9,0.3,0.2,0.0,-0.1,0.1,0.0,0.2,0.2,0.0,-0.1,-0.1,0.3,0.2,0.1,0.2,0.3,0.4,0.3,0.2,0.0,0.1,0.0,0.1,0.1,0.1,0.1,0.1,0.2,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.2,-0.1,-0.6,0.0,-0.3,0.0,0.2,1.3,0.3,0.9,2.7,1.9,0.2,-0.2,0.5,0.5,0.5,0.0,0.3,0.0,0.0,0.0,-0.1,0.0,-0.2,0.1,0.0,-0.1,-0.1,-0.1,0.3,0.

<div id="st-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Transpose the intermediate revisions dataset </span>
  </div>

In [19]:
# Use the function to transpose the intermediate revisions
inter_revisions_df = transpose_inter_revisions(inter_revisions_df)

# Display the result
inter_revisions_df.head(20)

,inter_revision_date,gdp_revision_1,gdp_revision_2,gdp_revision_3,gdp_revision_4,gdp_revision_5,gdp_revision_6,gdp_revision_7,gdp_revision_8,gdp_revision_9,gdp_revision_10,gdp_revision_11,gdp_revision_12,gdp_revision_13,gdp_revision_14,gdp_revision_15,gdp_revision_16,gdp_revision_17,gdp_revision_18,gdp_revision_19,gdp_revision_20
0,sep_2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,oct_2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nov_2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dic_2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ene_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,feb_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mar_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,abr_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,may_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,jun_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
inter_revisions_df['inter_revision_date'].dtype

dtype('O')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-1" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-1" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3.2.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Clean-up intermediate revisions dataset (last version to be loaded to SQL)
    </span>
    </h2>

In [21]:
# Extract the month and year from the 'inter_revision_date' column
inter_revisions_df['month'] = inter_revisions_df['inter_revision_date'].str.split('_').str[0]
inter_revisions_df['year'] = inter_revisions_df['inter_revision_date'].str.split('_').str[1]

# Map the month names to their respective numbers
month_mapping = {
    'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04',
    'may': '05', 'jun': '06', 'jul': '07', 'ago': '08',
    'sep': '09', 'oct': '10', 'nov': '11', 'dic': '12'
}

inter_revisions_df['month'] = inter_revisions_df['month'].map(month_mapping)

# Create a new column with the date in YYYY-MM-DD format
inter_revisions_df['inter_revision_date'] = inter_revisions_df['year'] + '-' + inter_revisions_df['month']

# Convert the 'inter_revision_date' column to datetime type
inter_revisions_df['inter_revision_date'] = pd.to_datetime(inter_revisions_df['inter_revision_date'], format='%Y-%m')

# Remove the temporary 'month' and 'year' columns
inter_revisions_df.drop(['month', 'year'], axis=1, inplace=True)

# Convert the 'inter_revision_date' column to datetime (keeping it as datetime64)
inter_revisions_df['inter_revision_date'] = pd.to_datetime(inter_revisions_df['inter_revision_date'], errors='coerce')

# Apply the function to convert columns to float and round
inter_revisions_df = convert_to_float_and_round(inter_revisions_df)

# Display the result
inter_revisions_df

,inter_revision_date,gdp_revision_1,gdp_revision_2,gdp_revision_3,gdp_revision_4,gdp_revision_5,gdp_revision_6,gdp_revision_7,gdp_revision_8,gdp_revision_9,gdp_revision_10,gdp_revision_11,gdp_revision_12,gdp_revision_13,gdp_revision_14,gdp_revision_15,gdp_revision_16,gdp_revision_17,gdp_revision_18,gdp_revision_19,gdp_revision_20
0,2011-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2012-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2012-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2012-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
print(inter_revisions_df['inter_revision_date'].dtype)

datetime64[ns]


In [24]:
print(inter_revisions_df['gdp_revision_1'].dtype)

float64


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: charter;">4.</span>
    <span style = "color: dark; font-family: charter;">
    Loading to SQL
    </span>
    </h2>

In [ ]:
# Check if all environment variables are defined
if not all([host, user, password]):
    raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# REVISIONES

#horizon_df.to_sql(f'{sector}_{frequency}_growth_rates_horizon', engine, index=False, if_exists='replace')
#inter_revisions_df.to_sql(f'{sector}_{frequency}_inter_revisions', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#select" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#select" style="color: rgb(255, 32, 78); text-decoration: none;">Back to economic sector.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>